# Esercitazione 3

### Machine Learning

2025/03/27

## Struttura della soluzione generale

1. Viene creata una funzione `nested_cv` per automatizzare il processo di inner e outer cross-validation.
2. Si chiama la funzione del modello in questione, specificando la lista dei parametri in modo da utilizzare poi il metodo `nested_cv`.
3. Dato che i modelli riportano quasi le stesse metriche di valutazione, viene misurato anche il tempo di esecuzione dei processi.

In [1]:
import numpy as np
from time import time

#### Implementazione di `nested_cv`

Il metodo `nested_cv` ottimizza gli iperparametri mediante la `GridSearchCV` nella parte interna e valuta il modello sulla parte esterna del dataset.

Per ogni iterazione, calcola le metriche di valutazione richieste: $R^2$, $MAE$ (Mean Absolute Error) e $RMSE$ (Root Mean Squared Error) restituendo la *media* e la *deviazione standard* di ciascuna metrica. 

Parametri della funzione `nested_cv`:
- `model`: il modello di regressione in input;
- `param_grid`: il dizionario degli iperparametri per `GridSearchCV`;
- `X, y`: feature e variabile target;
- `outer_splits`: il numero di fold per la Cross-Validation Esterna (`default = 5`);
- `inner_splits`: il numero di fold per la Cross-Validation Interna (`default = 5`);
- `scoring`: lista delle metriche per la valutazione;
- `random_state`: seed per la riproducibilità dell'esperimento.

Output del metodo:
- Il dizionario con la media e la deviazione standard di ciascuna misura di valutazione.

In [2]:
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import r2_score, root_mean_squared_error, mean_absolute_error

def nested_cv(model, param_grid, X, y, outer_splits = 5,
              inner_splits = 5, scoring = ['r2'], random_state = 42):

    # Outer Cross Validation
    outer_cv = KFold(n_splits = outer_splits, shuffle = True, random_state = random_state)
    
    score_results = {metric: [] for metric in scoring}

    for train_idx, test_idx in outer_cv.split(X):
        
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # Inner Cross Validation
        inner_cv = KFold(n_splits = inner_splits, shuffle = True, random_state = random_state)
        grid_search = GridSearchCV(model, param_grid, cv = inner_cv,
                                   n_jobs = -1, scoring = scoring[0])
        grid_search.fit(X_train, y_train)

        best_model = grid_search.best_estimator_

        y_pred = best_model.predict(X_test)

        if 'r2' in scoring:
            score_results['r2'].append(r2_score(y_test, y_pred))
            
        if 'mae' in scoring:
            score_results['mae'].append(mean_absolute_error(y_test, y_pred))
            
        if 'rmse' in scoring:
            score_results['rmse'].append(root_mean_squared_error(y_test, y_pred))

    result = {}

    for metric, scores in score_results.items():
        result[f"Nested CV {metric.upper()}"] = f"{np.mean(scores):.4f} ± {np.std(scores):.4f}"

    return result

Nested Cross-Validation:

> https://machinelearningmastery.com/nested-cross-validation-for-machine-learning-with-python/

#### Scaling del dataset

Il dataset scelto per l'esercitazione è `california_hounsing`, utile per i modelli di regressione. Si è scelto di utilizzare uno `StandardScaler` per normalizzare i dati.

In [3]:
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

X, y = fetch_california_housing(return_X_y=True)

scaler = StandardScaler()
X_std = scaler.fit_transform(X) # Normalizzazione

## Esercizio 1

Confronto di più regressori della stessa categoria (quelli base, con regolarizzazione, robusti) su un dataset concordato con Cross Validation (k-Fold Validation come esterna, GridSearch come interna) e fine tuning degli iperparametri. 

Le metriche da prendere in considerazione sono $R^2$ e $RMSE$.

**Scelta della categoria di Regressori**

I regressori che utilizzano la **regolarizzazione** scelti:
* Ridge
* Lasso
* ElasticNet
* SGDRegressor

### Ridge Regression

La formula della Ridge Regression è la seguente:
$$
J(\mathbf{w})= \text{MSE}(\mathbf{w})+\alpha\dfrac{1}{2}\sum_i w_i^2
$$

La regolarizzazione $\mathcal{l}_2$ è la sfera rappresentata nell'immagine sotto.

<center><img src = 'https://raw.githubusercontent.com/rasbt/python-machine-learning-book-3rd-edition/master/ch04/images/04_05.png' width = 500></center>

In [4]:
from sklearn.linear_model import Ridge

t0 = time()

ridge_model = Ridge()

ridge_params = {
    'alpha': [1, 0.1, 0.01, 0.001, 0.0001], 
    'fit_intercept': [True, False],
    'solver': ['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
}

ridge_results = nested_cv(ridge_model, ridge_params, X_std, y, 
                          outer_splits = 5, inner_splits = 5, 
                          scoring = ['r2', 'rmse'])

print(f'Ridge Regression Results: \n {ridge_results}')
print(f'Training Ridge model in {(time() - t0):.3f}s')

Ridge Regression Results: 
 {'Nested CV R2': '0.6014 ± 0.0170', 'Nested CV RMSE': '0.7282 ± 0.0150'}
Training Ridge model in 39.131s


### Lasso Regression

La formula della Ridge Regression è la seguente:
$$
J(\mathbf{w}) = \text{MSE}(\mathbf{w}) + \alpha  \sum_i |w|_i =  \dfrac{1}{2N} \|y - \mathbf{Xw}\|^2_2 + \alpha  \|\mathbf{w}\|_1
$$

La regolarizzazione $\mathcal{l}_1$ è il quadrato ruotato rappresentato in figura.

<center><img src = 'https://raw.githubusercontent.com/rasbt/python-machine-learning-book-3rd-edition/master/ch04/images/04_06.png' width = 500></center>

In [5]:
from sklearn.linear_model import Lasso

t0 = time()

lasso_model = Lasso() # max_iter = 1000

lasso_params = {
    'alpha': [1, 0.1, 0.01, 0.001, 0.0001], 
    'fit_intercept': [True, False]}

lasso_results = nested_cv(lasso_model, lasso_params, X_std, y, 
                          outer_splits=5, inner_splits=5, 
                          scoring=['r2', 'rmse'])

print(f'Lasso Regression Results: \n {lasso_results}')
print(f'Trained Lasso model in {(time() - t0):.3f}s')

Lasso Regression Results: 
 {'Nested CV R2': '0.6023 ± 0.0176', 'Nested CV RMSE': '0.7274 ± 0.0155'}
Trained Lasso model in 1.978s


### ElasticNet

La ElasticNet unisce le regolarizzazioni della Ridge Regression e della Lasso.

$$\min_\mathbf{w} \frac{1}{N}\|\mathbf{Xw}-y\|^2_2 + \alpha\rho\|\mathbf{w}\|_1 + \frac{1}{2}\alpha(1-\rho)\|\mathbf{w}\|_2$$

Il parametro $\alpha$ è della regolarizzazione $\mathcal{l}_2$, mentre $\rho$ è la penalizzazione della $\mathcal{l}_1$.

In [6]:
from sklearn.linear_model import ElasticNet

t0 = time()

en_model = ElasticNet()

en_params = {
    'alpha': [1, 0.1, 0.01, 0.001, 0.0001], 
    'fit_intercept': [True, False]}

en_results = nested_cv(en_model, en_params, X_std, y, 
                       outer_splits = 5, inner_splits = 5, 
                       scoring = ['r2', 'rmse'], random_state = 42)

print(f'ElasticNet Regression Results: \n {en_results}')
print(f'Trained ElasticNet model in {(time() - t0):.3f}s')

ElasticNet Regression Results: 
 {'Nested CV R2': '0.6022 ± 0.0177', 'Nested CV RMSE': '0.7274 ± 0.0156'}
Trained ElasticNet model in 1.930s


### SGD Regression

La regressione mediante discesa del gradiente stocastica è un metodo iterativo.

In [7]:
from sklearn.linear_model import SGDRegressor

t0 = time()

SGD_model = SGDRegressor()

sgd_params = {
    'loss': ['squared_error', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'],
    'alpha': [1, 0.1, 0.01, 0.001, 0.0001],
    'penalty': ['l2', 'l1', 'elasticnet', None],
    'fit_intercept': [True, False]
}

sgd_results = nested_cv(SGD_model, sgd_params, X_std, y, 
                        outer_splits = 5, inner_splits = 5, 
                        scoring = ['r2', 'rmse'], random_state = 42)

print(f'SGD Regression Results: \n {sgd_results}')
print(f'Trained SGD model in {(time() - t0):.3f}s')

SGD Regression Results: 
 {'Nested CV R2': '0.5687 ± 0.0187', 'Nested CV RMSE': '0.7575 ± 0.0168'}
Trained SGD model in 13.462s


## Esercizio 2

Confronto di più regressori di categorie diverse su un dataset concordato con Cross Validation e fine tuning degli iperparametri.

Le metriche da prendere in considerazione sono: $R^2$ e $MAE$.

**Scelta dei Regressori**

I regressori scelti per il confronto sono i seguenti:
* Linear
* RANSAC
* Bayesian Ridge

### Linear Regression

In [8]:
from sklearn.linear_model import LinearRegression

t0 = time()

linear_model = LinearRegression()

linear_param = {
    'fit_intercept': [True, False] 
}

linear_results = nested_cv(linear_model, linear_param, X_std, y, 
                           outer_splits = 5, inner_splits = 5, 
                           scoring = ['r2', 'mae'])

print(f'Linear Regression Results: \n {linear_results}')
print(f'Trained linear model in {(time() - t0):.3f}s')

Linear Regression Results: 
 {'Nested CV R2': '0.6014 ± 0.0170', 'Nested CV MAE': '0.5317 ± 0.0084'}
Trained linear model in 1.013s


### RANSAC Regressor

In [9]:
from sklearn.linear_model import RANSACRegressor

t0 = time()

ransac_model = RANSACRegressor()

ransac_params = {
    'min_samples': [0.5, 0.7, 0.9],  
    'residual_threshold': [5.0, 10.0, 15.0],  
    'loss': ['absolute_error', 'squared_error']
}

ransac_results = nested_cv(ransac_model, ransac_params, X_std, y, 
                           outer_splits = 5, inner_splits = 5, 
                           scoring = ['r2', 'mae'], random_state = 42)

print(f'RANSAC Regression Results: \n {ransac_results}')
print(f'Trained RANSAC model in {(time() - t0):.3f}s')

RANSAC Regression Results: 
 {'Nested CV R2': '0.6023 ± 0.0166', 'Nested CV MAE': '0.5307 ± 0.0081'}
Trained RANSAC model in 19.615s


### Bayesian Ridge

In [10]:
from sklearn.linear_model import BayesianRidge

t0 = time()

bayesian_ridge_model = BayesianRidge()

bayesian_param = {
    'alpha_1': [1e-6, 1e-3, 1e-1, 1],
    'alpha_2': [1e-6, 1e-3, 1e-1, 1],
    'lambda_1': [1e-6, 1e-3, 1e-1, 1],
    'lambda_2': [1e-6, 1e-3, 1e-1, 1]
}

bayesian_ridge_results = nested_cv(bayesian_ridge_model, bayesian_param, X_std, y, 
                                   outer_splits = 5, inner_splits = 5, scoring = ['r2', 'mae'])

print(f'Bayesian Ridge Regression Results: \n {bayesian_ridge_results}')
print(f'Trained Bayesian Ridge model in {(time() - t0):.3f}s')

Bayesian Ridge Regression Results: 
 {'Nested CV R2': '0.6014 ± 0.0170', 'Nested CV MAE': '0.5317 ± 0.0084'}
Trained Bayesian Ridge model in 11.508s
